<a href="https://colab.research.google.com/github/derrickaw/beam/blob/updateYamlNotebook/Copy_of_Try_Apache_Beam_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Try Apache Beam - YAML

While Beam provides powerful APIs for authoring sophisticated data processing pipelines, it still has a high barrier for getting started and authoring simple pipelines. Even setting up the environment, installing the dependencies, and setting up the project can be a challenge.

Here we provide a simple declarative syntax for describing pipelines that does not require coding experience or learning how to use an SDK&mdash;any text editor will do. Some installation may be required to actually *execute* a pipeline, but we envision various services (such as Dataflow) to accept yaml pipelines directly obviating the need for even that in the future. We also anticipate the ability to generate code directly from these higher-level yaml descriptions, should one want to graduate to a full Beam SDK (and possibly the other direction as well as far as possible).

It should be noted that everything here is still under development, but any features already included are considered stable. Feedback is welcome at dev@apache.beam.org.

In this notebook, you set up your development environment and write a simple pipeline using YAML. Then you run it locally, using the [DirectRunner](https://beam.apache.org/documentation/runners/direct/). You can explore other runners with the [Beam Capability Matrix](https://beam.apache.org/documentation/runners/capability-matrix/).

To navigate through different sections, use the table of contents. From **View**  drop-down list, select **Table of contents**.

To run a code cell, click the **Run cell** button at the top left of the cell, or select it and press **`Shift+Enter`**. Try modifying a code cell and re-running it to see what happens.

To learn more about Colab, see [Welcome to Colaboratory!](https://colab.sandbox.google.com/notebooks/welcome.ipynb).

# Setup

First, you need to set up your environment. The following code installs `apache-beam` and creates directories for your data, pipelines and results.

In [ ]:
# Install apache-beam
%pip install --quiet apache-beam[yaml,gcp]==2.71.0rc3
!gcloud auth login derrickaw@google.com
!gcloud config set project google.com:clouddfe
!gcloud config get-value project
!gcloud auth application-default login
!gcloud auth application-default set-quota-project google.com:clouddfe
# !gcloud auth application-default login --quiet


# Create directories for storing data, pipelines and results
! mkdir -p data
! mkdir -p pipelines
! mkdir -p results


You are now logged in as [derrickaw@google.com].
Your current project is [google.com:clouddfe].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
google.com:clouddfe
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=b7c5kvUAJiJm5krWUC2hKZNVbmVVAr&prompt=consent&token_usage=remote&access_type=offline&code_challenge=xNjuhtIBqptmNFTtpKSZmcr05Gv308YKbypfIOOPDVY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ASc3gC1dZ3aW2jFCwWk0t2fI3

We'll also create an artificial dataset that represents a simple database. The csv data contains information about different people. Each line represents a single person and their details are separated by commas. The file contains 5 columns: id, firstname, age, country and a profession.

In [ ]:
%%writefile 'data/people.csv'
id,firstname,age,country,profession
1,Reeba,58,Belgium,unemployed
2,Maud,45,Spain,firefighter
3,Meg,11,France,unemployed
4,Rani,53,Spain,doctor
5,Natka,26,France,doctor
6,Aurore,32,Italy,police officer
7,Elvira,39,Italy,doctor
8,Asia,10,Belgium,doctor
9,Lesly,35,Spain,firefighter
10,Orelia,31,Germany,police officer
11,Theodora,16,Italy,unemployed
12,Viviene,44,Germany,police officer
13,Teriann,50,Belgium,police officer
14,Carol-Jean,23,Germany,unemployed
15,Drucie,15,Spain,police officer
16,Elie,10,Italy,doctor
17,Shaylyn,34,Belgium,worker
18,Fayre,33,Spain,police officer
19,Sabina,52,Germany,police officer
20,Aryn,20,Belgium,police officer
21,Darlleen,49,Spain,worker
22,Jere,18,Italy,worker
23,Candi,60,Germany,police officer
24,Sindee,40,Germany,firefighter
25,Selma,20,Spain,worker
26,Vonny,35,Germany,doctor
27,Kate,53,Spain,worker
28,Annabela,48,Belgium,worker
29,Jenilee,55,Germany,police officer
30,Gusella,44,France,police officer
31,Fawne,35,Spain,worker
32,Karolina,39,Spain,police officer
33,Sadie,58,Germany,firefighter
34,Clo,10,Italy,police officer
35,Beth,46,Spain,firefighter
36,Adore,18,Italy,firefighter
37,Tarra,29,Spain,doctor
38,Jessamyn,36,France,police officer
39,Deedee,24,Germany,unemployed
40,Patricia,45,Italy,doctor
41,Wileen,39,Spain,police officer
42,Paola,55,Italy,worker
43,Gwyneth,37,Italy,worker
44,Stacey,36,Spain,worker
45,Camile,60,Germany,unemployed
46,Sheree,10,Spain,unemployed
47,Albertina,53,France,police officer
48,Jinny,30,Spain,firefighter
49,Kayla,57,Italy,firefighter
50,Jaime,55,France,doctor

Overwriting data/people.csv


Let's validate if the file was created correctly. You should see the first few lines from the generated file. Validate if the beginning of the file matches with the declared content above.

In [ ]:
! head data/people.csv

id,firstname,age,country,profession
1,Reeba,58,Belgium,unemployed
2,Maud,45,Spain,firefighter
3,Meg,11,France,unemployed
4,Rani,53,Spain,doctor
5,Natka,26,France,doctor
6,Aurore,32,Italy,police officer
7,Elvira,39,Italy,doctor
8,Asia,10,Belgium,doctor
9,Lesly,35,Spain,firefighter


# Your first YAML pipelines

In this section we'll present you the basic structure of a YAML pipeline and present you some available transforms.
Below is a simple pipeline that reads data from the csv file we've just created and logs the elements for debugging purposes.

The `LogForTesting` transform lets us log the data when developing a pipeline. Remember, it is not advised to use this transform in production.

Let's define the pipeline and save it to a file:

In [ ]:
%%writefile 'pipelines/pipeline-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: LogForTesting

Overwriting pipelines/pipeline-01.yaml


We can verify the contents of this file by running:

In [ ]:
! cat pipelines/pipeline-01.yaml

pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: LogForTesting


Now, we can execute the yaml pipeline by passing this file as an argument to the following command:

In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-01.yaml --runner=DataflowRunner --project=google.com:clouddfe --region=us-central1 --temp_location=gs://derrickaw-test/temp --staging_location=gs://derrickaw-test/temp


Building pipeline...
2026-01-20 19:44:15.913679: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:44:15.918901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:44:15.934463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768938255.959552   38575 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768938255.966385   38575 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768938255.984019   38575 computation_placer.cc:177] computation placer already registered. Please check lin

Here we use Python and `apache_beam` package to execute the pipeline, but we envision various services (such as Dataflow) to accept yaml pipelines directly obviating the need for that in the future.

If you scroll through the output logs, you'll find entries such as:
```
INFO:root:BeamSchema_edf39b51_91da_418a_b28e_af04c9bae811(id=1, firstname='Reeba', age=58, country='Belgium', profession='unemployed')
INFO:root:BeamSchema_edf39b51_91da_418a_b28e_af04c9bae811(id=2, firstname='Maud', age=45, country='Spain', profession='firefighter')
INFO:root:BeamSchema_edf39b51_91da_418a_b28e_af04c9bae811(id=3, firstname='Meg', age=11, country='France', profession='unemployed')
INFO:root:BeamSchema_edf39b51_91da_418a_b28e_af04c9bae811(id=4, firstname='Rani', age=53, country='Spain', profession='doctor')
```
This is a representation of records from the input dataset.

Let's add a transform - `Filter`. To use this transform you need to  specify the 'keep' condition and a language your condition is written in. Below you'll find an example with a condition written in Python.
This pipeline will filter out records containing people that are younger than 18 years old. The only records left to the next transform will be records representing adults. Verify this by scrolling to the bottom of the output logs.

In [ ]:
%%writefile 'pipelines/pipeline-filter-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: Filter
      config:
        language: python
        keep: "age >= 18"
    - type: LogForTesting

Overwriting pipelines/pipeline-filter-01.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-filter-01.yaml --runner=PrismRunner

Building pipeline...
2026-01-20 19:51:23.984074: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:51:23.990548: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:51:24.020437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768938684.075062   40284 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768938684.090512   40284 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768938684.133356   40284 computation_placer.cc:177] computation placer already registered. Please check lin

Similarly, we can create a condition in other languages, for example SQL. In this example we filter out people that are younger than 18 and have a profession other than being 'unemployed'.

In [ ]:
%%writefile 'pipelines/pipeline-filter-02.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: Filter
      config:
        language: sql
        keep: "age >= 18 or (age < 18 and profession = 'unemployed')"
    - type: LogForTesting

Overwriting pipelines/pipeline-filter-02.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-filter-02.yaml --runner=PrismRunner

Building pipeline...
2026-01-20 19:55:11.926582: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:55:11.931889: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:55:11.951902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768938911.993817   41436 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768938912.004938   41436 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768938912.032182   41436 computation_placer.cc:177] computation placer already registered. Please check lin

You'll notice that the output of this pipeline is in a different format than the previous one. That's because this pipeline uses an SQL Filter transform, an example of a [multi-language transform](https://beam.apache.org/documentation/programming-guide/#multi-language-pipelines). Multi-language pipelines are an important feature of Beam, but in this notebook we'll focus on YAML.

To find the output of this pipeline find lines that begin with 'message' keyword and have the associated 'transform_id' set to a transform starting with 'LogForTesting'.
Example:
```
message: "{\"id\":49,\"firstname\":\"Kayla\",\"age\":57,\"country\":\"Italy\",\"profession\":\"firefighter\"}"
instruction_id: "bundle_6"
transform_id: "LogForTesting/beam:schematransform:org.apache.beam:yaml:log_for_testing:v1/LogAsJson/ParMultiDo(Anonymous)"
```
Each log entry represents one element from the output data.

Another useful transform is `MapToFields`. This transform lets us manipulate fields of a record. For example, we can add a field to our records, which is a boolean field specifying if the person is adult or not.

In [ ]:
%%writefile 'pipelines/pipeline-map-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: MapToFields
      config:
        language: python
        append: true
        fields:
          is_adult: "age >= 18"
    - type: LogForTesting


Overwriting pipelines/pipeline-map-01.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-map-01.yaml  --runner=DataflowRunner --project=google.com:clouddfe --region=us-central1 --temp_location=gs://derrickaw-test/temp --staging_location=gs://derrickaw-test/temp

Building pipeline...
2026-01-20 19:59:24.204834: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:59:24.210121: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 19:59:24.223802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768939164.246723   42734 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768939164.253449   42734 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768939164.271032   42734 computation_placer.cc:177] computation placer already registered. Please check lin

Beam will try to infer the types involved in the mappings, but sometimes this is not possible. In these cases we can explicitly denote the expected output type, e.g.

In [ ]:
%%writefile 'pipelines/pipeline-map-02.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: MapToFields
      config:
        language: python
        append: true
        fields:
           is_adult:
             expression: "age >= 18"
             output_type: boolean
    - type: LogForTesting

Writing pipelines/pipeline-map-02.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-map-02.yaml --runner=PrismRunner

Building pipeline...
2026-01-20 20:09:28.226903: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 20:09:28.234402: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 20:09:28.256140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768939768.293682   45220 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768939768.304832   45220 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768939768.334826   45220 computation_placer.cc:177] computation placer already registered. Please check lin

When the `append` field is specified, one can `drop` fields as well, e.g.

In [ ]:
%%writefile 'pipelines/pipeline-map-03.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: MapToFields
      config:
        language: python
        append: true
        fields:
          is_adult: "age >= 18"
        drop:
          - age
    - type: LogForTesting

Writing pipelines/pipeline-map-03.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-map-03.yaml

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
Building pipeline...
2026-01-20 18:04:32.570256: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:04:32.575158: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:04:32.588710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768932272.611562   13383 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768932272.619408   13383 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:17689

We can also create simple UDFs (User Defined Functions) using Python or other languages. In the example below we add a field `random_number` which value is a random number not bigger than the age of the person.

In [ ]:
%%writefile 'pipelines/pipeline-map-04.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: MapToFields
      config:
        language: python
        append: true
        fields:
          random_number:
            callable: |
              import random
              def my_mapping(row):
                return random.randrange(row.age)
    - type: LogForTesting

Writing pipelines/pipeline-map-04.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-map-04.yaml

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
Building pipeline...
2026-01-20 18:06:01.099794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:06:01.107670: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:06:01.137814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768932361.175862   13789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768932361.185337   13789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:17689

Beam YAML has the ability to do aggregations to group and combine values across records. The is accomplished via the `Combine` transform type.

In this example we'll aggregate our records based on the `is_adult` classification. We'll calculate an average age for each of the groups.

In [ ]:
%%writefile 'pipelines/pipeline-combine-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: MapToFields
      config:
        language: python
        append: true
        fields:
          is_adult: "age >= 18"
    - type: Combine
      config:
        group_by: is_adult
        combine:
          total:
            value: age
            fn: mean
    - type: LogForTesting

Writing pipelines/pipeline-combine-01.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-combine-01.yaml

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
Building pipeline...
2026-01-20 18:06:54.386725: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:06:54.391871: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 18:06:54.407244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768932414.430450   14062 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768932414.437055   14062 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:17689

If all was executed correctly, you should see the following lines at the bottom of the output log:
```
INFO:root:Result(is_adult=True, total=40.674418604651166)
INFO:root:Result(is_adult=False, total=11.714285714285714
```

All the previous pipelines were linear - output of one transform was an input to the next transform. This is also known as a `chain` pipeline. This is designated in the top-level pipeline configuration, for example:
```
pipeline:
  type: chain
  transforms:
    ...
```
In YAML we can also create nonlinear pipelines. To do this, we can specify `type: composite`, or omit this line completely (this is the default pipeline type). In these pipelines, we must specify the `input` in each of the transforms that take the output of previous transforms. This `input` is the name, or collection of names, of the transform(s) that feed into the receiving transform.
The specification below will create the following pipeline:
```
             +----> Doctors -----------> SaveDoctors
InputData ---+
             +----> OtherProfessions --> SaveOtherProfessions
```

In [ ]:
%%writefile pipelines/pipeline-nonlinear-01.yaml
pipeline:
  type: composite
  transforms:
    - type: ReadFromCsv
      name: InputData
      config:
        path: data/people.csv
    - type: Filter
      name: Doctors
      input: InputData
      config:
        language: python
        keep: "profession == 'doctor'"
    - type: Filter
      name: OtherProfessions
      input: InputData
      config:
        language: python
        keep: "profession != 'doctor'"
    - type: WriteToCsv
      name: SaveDoctors
      input: Doctors
      config:
        path: results/doctors
    - type: WriteToCsv
      name: SaveOtherProfessions
      input: OtherProfessions
      config:
        path: results/other-professions

Overwriting pipelines/pipeline-nonlinear-01.yaml


In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-nonlinear-01.yaml  --runner=DataflowRunner --project=google.com:clouddfe --region=us-central1 --temp_location=gs://derrickaw-test/temp --staging_location=gs://derrickaw-test/temp

Building pipeline...
2026-01-20 20:26:42.923901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 20:26:42.931428: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 20:26:42.945437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768940802.970388   49594 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768940802.977827   49594 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768940802.996256   49594 computation_placer.cc:177] computation placer already registered. Please check lin

The output are 2 files: `results/doctors-00000-of-00001` and `results/other-professions-00000-of-00001`. Let's see their contents:

In [ ]:
! head results/doctors-00000-of-00001

id,firstname,age,country,profession
4,Rani,53,Spain,doctor
5,Natka,26,France,doctor
7,Elvira,39,Italy,doctor
8,Asia,10,Belgium,doctor
16,Elie,10,Italy,doctor
26,Vonny,35,Germany,doctor
37,Tarra,29,Spain,doctor
40,Patricia,45,Italy,doctor
50,Jaime,55,France,doctor


In [ ]:
! head results/other-professions-00000-of-00001

id,firstname,age,country,profession
1,Reeba,58,Belgium,unemployed
2,Maud,45,Spain,firefighter
3,Meg,11,France,unemployed
6,Aurore,32,Italy,police officer
9,Lesly,35,Spain,firefighter
10,Orelia,31,Germany,police officer
11,Theodora,16,Italy,unemployed
12,Viviene,44,Germany,police officer
13,Teriann,50,Belgium,police officer


# Summary

Congratulations! You've just run Apache Beam pipelines using YAML.

To learn more about Beam YAML visit [Beam YAML API documentation page](https://beam.apache.org/documentation/sdks/yaml/).

For extra practice with different runners see below.


# Prism Runner

To run your pipelines with the newer Prism Runner just add this flag - `--runner=PrismRunner` - on any execution lines above and as noted in the below example.

Save the yaml pipeline again.

In [ ]:
%%writefile 'pipelines/pipeline-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: LogForTesting

Overwriting pipelines/pipeline-01.yaml


Execute with the newer Prism Runner.

In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-01.yaml --runner=PrismRunner

Building pipeline...
2026-01-20 21:44:58.884468: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 21:44:58.910055: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-20 21:44:58.961795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768945499.030458   68341 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768945499.059249   68341 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768945499.170677   68341 computation_placer.cc:177] computation placer already registered. Please check lin

# Dataflow Runner

To run your pipeline in Dataflow, you'll need to set up your Google Cloud and run the pipeline with the `--runner=DataflowRunner`. Fill in the required fields below to do the general setup.

For additional context, follow https://cloud.google.com/dataflow/docs/guides/deploying-a-pipeline#run-on-dataflow.

In [ ]:
%pip install --quiet apache-beam[gcp]
!gcloud auth login <YOUR_LOGIN>
!gcloud config set project <YOUR_GCP_PROJECT>
!gcloud auth application-default login
!gcloud auth application-default set-quota-project <YOUR_GCP_PROJECT>

Then save your pipeline.

In [ ]:
%%writefile 'pipelines/pipeline-01.yaml'
pipeline:
  type: chain
  transforms:
    - type: ReadFromCsv
      config:
        path: data/people.csv
    - type: LogForTesting

Then run your pipeline with the `--runner=DataflowRunner` flag and make sure to fill in the required other flags.

In [ ]:
! python -m apache_beam.yaml.main --pipeline_spec_file=pipelines/pipeline-01.yaml  --runner=DataflowRunner --project=<YOUR_GCP_PROJECT> --region=<YOUR_REGION> --temp_location=gs://<YOUR_TEMP_BUCKET> --staging_location=gs://<YOUR_STAGING_BUCKET>